In [1]:
import pandas as pd

np = pd.read_csv('node_populations.csv')

S0_24 = np['S(0-15)'] + np['S(16-24)']
S25_49 = np['S(25-34)'] + np['S(35-49)']
S50_75 = np['S(50-64)'] + np['S(65-74)']
S75p =  np['S(75+)']

np_4 = pd.DataFrame( {
    'Home' : np['Home'],
    'Location' : np['Location'],
    'S(0-24)' : S0_24,
    'I(0-24)' : 0,
    'R(0-24)' : 0,
    'S(25-49)' : S25_49,
    'I(25-49)' : 0,
    'R(25-49)' : 0,
    'S(50-79)' : S50_75,
    'I(50-79)' : 0,
    'R(50-79)' : 0,
    'S(75+)' : S75p,
    'I(75+)' : 0,
    'R(75+)' : 0
})

np_4.to_csv('node_populations_ag4.csv', index=False)

KeyError: 'S(0-15)'

In [ ]:
import pandas as pd

np = pd.read_csv('node_populations.csv')

S0_49 = S0_24 + S25_49
S50p = S50_75 + S75p

np_2 = pd.DataFrame( {
    'Home' : np['Home'],
    'Location' : np['Location'],
    'S(0-49)' : S0_49,
    'E(0-49)' : 0,
    'I(0-49)' : 0,
    'R(0-49)' : 0,
    'S(50+)' : S50p,
    'E(50+)' : 0,
    'I(50+)' : 0,
    'R(50+)' : 0,
})

np_2.to_csv('node_populations_ag2.csv', index=False)

In [ ]:
cn = pd.read_csv('commuter_networks.csv')

cn['Age'] = cn['Age'] - 1
cn2 = cn[cn['Age'] < 4]

cn.to_csv('commuter_networks_ag4.csv', index=False)

In [ ]:
cn2 = cn[cn['Age'] < 2]

cn2.loc[ cn2['Home'] == cn2['From'], 'Allow I' ] = 0

cn2.to_csv('commuter_networks_ag2.csv', index=False)

In [2]:
# Compile and import local geoSIR module
import os, sys
sys.path.insert(0,'../../')
import geodemic

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle, zarr

In [ ]:
node_mappings = pickle.load( open( 'node_mappings.pkl', "rb" ) )
cnode_mappings = pickle.load( open( 'cnode_mappings.pkl', "rb" ) )
ts_saved = np.load('ts.npy')
X_states = zarr.open('X_states.zarr')[:]

ts = np.load('ts.npy')
location_data = np.load('location_data.npy')
ts_days = ts / (60*24)

# Load model

model_classes = []

import json
with open('system_parameters/model.json', 'r') as json_file:
    model_dat = json.load(json_file)
    model_classes = model_dat['classes']

In [ ]:
network_data = np.sum( location_data, axis=3 )

t1 = np.min(ts_days)
t2 = np.max(ts_days)
s = (ts_days>=t1) * (ts_days<=t2)

plot_classes = model_classes
for oi, Oclass in zip(range(len(model_classes)), model_classes):
    if Oclass in plot_classes:
        O = np.sum(network_data[:, :, oi], axis=1)
        plt.plot(ts_days[s], O[s], label=Oclass)
    
plt.legend(loc='upper right', fontsize=12)

In [ ]:
plt.figure( figsize=(8,3) )

loc = 504

t1 = np.min(ts_days)
t2 = 4#np.max(ts_days)
s = (ts_days>=t1) * (ts_days<=t2)

class_data = []
N = np.zeros( location_data.shape[0])
for oi, Oclass in zip(range(len(model_classes)), model_classes):
    O = np.sum(location_data[:,:,oi,loc], axis=1)
    N += O
    class_data.append(O)
    
plot_classes = model_classes
for Oclass in plot_classes:
    O = class_data[model_classes.index(Oclass)]
    plt.plot(ts_days[s], O[s], label=Oclass)
    
#plt.plot(ts_days[s], N[s], label="$N$")

plt.legend(loc='upper right', fontsize=12)